In [1]:
import os
import glob
import cv2
import random
import pandas as pd
from skimage import io
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import h5py

# Network building stuff
import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import torchmetrics

In [2]:
import sys
sys.path.append("/home2/sdokania/all_projects/project-noisypixel/")

In [3]:
from src.models import *
from src.dataset.dataloader import OccupancyNetDatasetHDF
from src.trainer import ONetLit
from src.utils import Config, count_parameters

In [6]:
config = Config()

In [7]:
onet = ONetLit(config)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home2/sdokania/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [9]:
# onet

In [17]:
f = open("/ssd_scratch/cvit/sdokania/Shapenet3D/ShapeNet/02691156/test.lst", "r")
data = ["{}_{}.hdf5".format(1234, ix) for ix in f.read().split()[:10]]
f.close()
print(data)

['1234_d18592d9615b01bbbc0909d98a1ff2b4.hdf5', '1234_d18f2aeae4146464bd46d022fd7d80aa.hdf5', '1234_d199612c22fe9313f4fb6842b3610149.hdf5', '1234_d1a887a47991d1b3bc0909d98a1ff2b4.hdf5', '1234_d1a8e79eebf4a0b1579c3d4943e463ef.hdf5', '1234_d1b1c13fdec4d69ccfd264a25791a5e1.hdf5', '1234_d1b28579fde95f19e1873a3963e0d14.hdf5', '1234_d1b407350e61150942d79310bc7e47b3.hdf5', '1234_d1cdd239dcbfd018bbf3143b1cb6076a.hdf5', '1234_d1d308692cb4b6059a6e43b878d5b335.hdf5']


In [26]:
class OccupancyNetDatasetHDF(Dataset):
    """Occupancy Network dataset."""

    def __init__(self, root_dir, transform=None, num_points=1024, default_transform=True, mode="train"):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
            num_points (int): Number of points to sample in the object point cloud from the data
                on a sample.
            mode (str): Which data split do we want among train, test and val
        """
        self.root_dir = root_dir
        self.transform = transform
        self.num_points = num_points
        self.mode = mode
        self.files = []
        
        # Save the files
        f = open(os.path.join(self.root_dir, "{}.lst".format(self.mode)), 'r')
        self.files = f.read().split()
        f.close()
            
        # If not transforms have been provided, apply default imagenet transform
        if transform is None and default_transform:
            self.transform = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                  std=[0.229, 0.224, 0.225])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        # Fetch the file path and setup image folder paths
        req_path = self.files[idx]
        file_path = os.path.join(self.root_dir, req_path)

        # Load the h5 file
        hf = h5py.File(file_path, 'r')
        
        # [NOTE]: the notation [()] below is to extract the value from HDF5 file
        # get all images and randomly pick one
        all_imgs = hf['images'][()]
        random_idx = int(np.random.random()*all_imgs.shape[0])
        
        # Fetch the image we need
        image = all_imgs[random_idx]
        
        # Get the points and occupancies
        points = hf['points']['points'][()]
        occupancies = np.unpackbits(hf['points']['occupancies'][()])

        # Sample n points from the data
        selected_idx = np.random.permutation(np.arange(points.shape[0]))[:self.num_points]

        # Use only the selected indices and pack everything up in a nice dictionary
        final_image = torch.from_numpy(image).float().transpose(1, 2).transpose(0, 1) / image.max()
        final_points = torch.from_numpy(points[selected_idx]).float()
        final_gt = torch.from_numpy(occupancies[selected_idx]).float()
        
        # Close the hdf file
        hf.close()
        
        # Apply any transformation necessary
        if self.transform:
            final_image = self.transform(final_image)

        return [final_image, final_points, final_gt]

In [27]:
data_root = "/ssd_scratch/cvit/sdokania/hdf_shapenet/hdf_data/"

In [28]:
train_df = OccupancyNetDatasetHDF(data_root)

In [29]:
op = train_df[0]

In [31]:
op[0].shape, op[1].shape, op[2].shape

(torch.Size([3, 137, 137]), torch.Size([1024, 3]), torch.Size([1024]))